In [2]:
import spacy

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.probability import FreqDist
from nltk.tokenize.treebank import TreebankWordDetokenizer
import pprint
from heapq import nlargest

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("corpus")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Anton\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anton\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading corpus: Package 'corpus' not found in index


False

In [3]:
text_to_summarize = "The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering. Because it has achieved significance within the past fifty years, Criteria Consideration G applies. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA. Unlike the Mercury, Gemini, and Apollo programs, the SSP’s emphasis was on cost effectiveness and reusability, and eventually the construction of a space station. Including her maiden voyage (launched August 30, 1984), Discovery flew to space thirty-nine times, more than any of the other four orbiters; she was also the first orbiter to fly twenty missions. She had the honor of being chosen as the Return to Flight vehicle after both the Challenger and Columbia accidents. Discovery was the first shuttle to fly with the redesigned SRBs, a result of the Challenger accident, and the first shuttle to fly with the Phase II and Block I SSME. Discovery also carried the Hubble Space Telescope to orbit and performed two of the five servicing missions to the observatory. She flew the first and last dedicated Department of Defense (DoD) missions, as well as the first unclassified defense-related mission. In addition, Discovery was vital to the construction of the International Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to the station by a U.S. Space Shuttle. She was the first orbiter to dock to the ISS, and the first to perform an exchange of a resident crew. Under Criterion C, Discovery is significant as a feat of engineering. According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.” Although her base structure followed a conventional aircraft design, she used advanced materials that both minimized her weight for cargo-carrying purposes and featured low thermal expansion ratios, which provided a stable base for her Thermal Protection System (TPS) materials. The Space Shuttle orbiter also featured the first reusable TPS; all previous spaceflight vehicles had a single-use, ablative heat shield. Other notable engineering achievements of the orbiter included the first reusable orbital propulsion system, and the first two-fault-tolerant Integrated Avionics System. As Hale stated, the Space Shuttle remains “the largest, fastest, winged hypersonic aircraft in history,” having regularly flown at twenty-five times the speed of sound."

In [4]:
len(text_to_summarize)

2906

Нижче моя функція суммарізації тексту NLTK. компресія це % ключоваих слів в кожному реченні (вибирається 10 найпоширениших) і другий параметр - мінімальна кількість речень в резюме тексту. 

In [90]:
def nltk_summary(text, compression=0.4, min_sentences=5):
    # Tokenize sentences
    sentences = sent_tokenize(text)

    # Tokenize words
    words = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words("english"))

    filtered_words = [
        word.lower()
        for word in words
        if word.isalnum() and word.lower() not in stop_words
    ]

    # Calculate word frequencies
    freq_dist = FreqDist(filtered_words)

    # Get the most frequent words
    top_words = freq_dist.most_common(10)

    common_words = [word for word, freq in top_words]
    
    # Score sentences
    sent_scores = {}
    for sentence in sentences:
        word_count = 0
        for word in common_words:
            if word in sentence.lower():
                word_count += 1
            sent_scores.update({sentence: word_count})

    # summarize text
    summary_sent = []
    target = len(top_words) * compression

    for sentence, score in sent_scores.items():
        if score >= target or len(summary_sent) < min_sentences:
            summary_sent.append(sentence)

    summary_text = " ".join(summary_sent)
    return summary_text, common_words


# Example usage
nltk_summary_result, words = nltk_summary(text_to_summarize, compression=0.6, min_sentences=2)
pprint.pprint(nltk_summary_result)
print(len(nltk_summary_result))

('The Orbiter Discovery, OV-103, is considered eligible for listing in the '
 'National Register of Historic Places (NRHP) in the context of the U.S. Space '
 'Shuttle Program (1969-2011) under Criterion A in the areas of Space '
 'Exploration and Transportation and under Criterion C in the area of '
 'Engineering. Because it has achieved significance within the past fifty '
 'years, Criteria Consideration G applies. Under Criterion A, Discovery is '
 'significant as the oldest of the three extant orbiter vehicles constructed '
 'for the Space Shuttle Program (SSP), the longest running American space '
 'program to date; she was the third of five orbiters built by NASA.')
644


Spacy

In [81]:
def spacy_summary(text):
    # Load SpaCy English model
    nlp = spacy.load("en_core_web_sm")

    # Process the text using SpaCy
    doc = nlp(text)
    
    # Get sentences
    sentences = [sent.text for sent in doc.sents]
    
       
    # Extract important sentences based on the number of tokens
    important_sentences = [sent for sent in sentences if len(sent.split(' ')) > 25]
    
    # Generate summary
    summary_text = " ".join(important_sentences)

    return summary_text


spacy_summary_result = spacy_summary(text_to_summarize)
pprint.pprint(spacy_summary_result)
print(len(spacy_summary_result))

('The Orbiter Discovery, OV-103, is considered eligible for listing in the '
 'National Register of Historic Places (NRHP) in the context of the U.S. Space '
 'Shuttle Program (1969-2011) under Criterion A in the areas of Space '
 'Exploration and Transportation and under Criterion C in the area of '
 'Engineering. Under Criterion A, Discovery is significant as the oldest of '
 'the three extant orbiter vehicles constructed for the Space Shuttle Program '
 '(SSP), the longest running American space program to date; she was the third '
 'of five orbiters built by NASA. Including her maiden voyage (launched August '
 '30, 1984), Discovery flew to space thirty-nine times, more than any of the '
 'other four orbiters; she was also the first orbiter to fly twenty missions. '
 'Discovery was the first shuttle to fly with the redesigned SRBs, a result of '
 'the Challenger accident, and the first shuttle to fly with the Phase II and '
 'Block I SSME. In addition, Discovery was vital to the co